# Executer du R depuis python (sans ouvrir R!)

rpy2 est un module qui permet d'executer du R depuis un script python.

Doc:
http://pandas.pydata.org/pandas-docs/stable/r_interface.html

L'idée est simple:
  1. On manipule des pandas.DataFrame
  2. On envois ce dataframe dans R via pandas2ri.py2ri. On obtient un *proxy* du datafarme de R.
  3. Un applioque un modele via un commande R sur ce DataFrame proxy.



In [2]:
import pandas as pd
import statsmodels as sm

from rpy2.robjects.packages import importr
from rpy2.robjects import r as R
from rpy2.robjects import pandas2ri
from rpy2.robjects import Formula
pandas2ri.activate()

In [4]:
# On va bosser sur le même DataFrame que statsmodels.
import pandas as pd
#ça marche plus ??
#df = sm.datasets.get_rdataset("Guerry", "HistData").data
#df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
#df.head()
df = pd.read_csv('./data/Guerry.csv')
df.head()
df = df[['Lottery', 'Literacy', 'Wealth', 'Region']].dropna()
df.head()

,Lottery,Literacy,Wealth,Region
0,41,37,73,E
1,38,51,22,N
2,66,13,61,C
3,80,46,76,E
4,79,69,83,E


# On met notre df dans R

In [5]:
rdf = pandas2ri.DataFrame(df)
#print(rdf)
print(type(rdf))

<class 'rpy2.robjects.vectors.DataFrame'>


In [6]:
rdf.head()

,Lottery,Literacy,Wealth,Region
0,41,37,73,E
1,38,51,22,N
2,66,13,61,C
3,80,46,76,E
4,79,69,83,E
5,70,27,84,S


# On import les packages R dont on a besoin

In [7]:
#r_base = importr('base')
importr('stats')

rpy2.robjects.packages.Package as a <module 'stats'>

# On applique le modele avec les même formules

In [8]:
formula = Formula("Lottery ~ Literacy")
#formula = "Lottery ~ Literacy"

#Formula() est facultatif ici mais ce n'est pas tout le temps le cas.

mod = R.lm(formula=formula, data=df)
print(R.summary(mod))





Call:
(function (formula, data, subset, weights, na.action, method = "qr", 
    model = TRUE, x = FALSE, y = FALSE, qr = TRUE, singular.ok = TRUE, 
    contrasts = NULL, offset, ...) 
{
    ret.x <- x
    ret.y <- y
    cl <- match.call()
    mf <- match.call(expand.dots = FALSE)
    m <- match(c("formula", "data", "subset", "weights", "na.action", 
        "offset"), names(mf), 0L)
    mf <- mf[c(1L, m)]
    mf$drop.unused.levels <- TRUE
    mf[[1L]] <- quote(stats::model.frame)
    mf <- eval(mf, parent.frame())
    if (method == "model.frame") 
        return(mf)
    else if (method != "qr") 
        warning(gettextf("method = '%s' is not supported. Using 'qr'", 
            method), domain = NA)
    mt <- attr(mf, "terms")
    y <- model.response(mf, "numeric")
    w <- as.vector(model.weights(mf))
    if (!is.null(w) && !is.numeric(w)) 
        stop("'weights' must be a numeric vector")
    offset <- model.offset(mf)
    mlm <- is.matrix(y)
    ny <- if (mlm) 
        nrow(y)
   

In [9]:
formula = Formula("Lottery ~ Literacy + Wealth + Region")

full_mod = R.lm(formula=formula, data=df)
print(R.summary(full_mod))



Call:
(function (formula, data, subset, weights, na.action, method = "qr", 
    model = TRUE, x = FALSE, y = FALSE, qr = TRUE, singular.ok = TRUE, 
    contrasts = NULL, offset, ...) 
{
    ret.x <- x
    ret.y <- y
    cl <- match.call()
    mf <- match.call(expand.dots = FALSE)
    m <- match(c("formula", "data", "subset", "weights", "na.action", 
        "offset"), names(mf), 0L)
    mf <- mf[c(1L, m)]
    mf$drop.unused.levels <- TRUE
    mf[[1L]] <- quote(stats::model.frame)
    mf <- eval(mf, parent.frame())
    if (method == "model.frame") 
        return(mf)
    else if (method != "qr") 
        warning(gettextf("method = '%s' is not supported. Using 'qr'", 
            method), domain = NA)
    mt <- attr(mf, "terms")
    y <- model.response(mf, "numeric")
    w <- as.vector(model.weights(mf))
    if (!is.null(w) && !is.numeric(w)) 
        stop("'weights' must be a numeric vector")
    offset <- model.offset(mf)
    mlm <- is.matrix(y)
    ny <- if (mlm) 
        nrow(y)
   

# anova simple

In [10]:
table = R.anova(full_mod)
print(table)
#print(R.summary(table))

           Df        Sum Sq      Mean Sq    F value    Pr(>F)
Literacy    1   7492.792767  7492.792767  17.168224  0.000086
Wealth      1   8384.268611  8384.268611  19.210861  0.000036
Region      4   1499.998582   374.999646   0.859236  0.492304
Residuals  78  34041.834158   436.433771        NaN       NaN


# anova avec interaction

In [11]:
formula = Formula("Lottery ~ Literacy * Wealth * Region")
mod = R.lm(formula=formula, data=df)
table = R.anova(mod)
print(table)

                        Df        Sum Sq      Mean Sq    F value    Pr(>F)
Literacy                 1   7492.792767  7492.792767  15.626415  0.000193
Wealth                   1   8384.268611  8384.268611  17.485611  0.000088
Region                   4   1499.998582   374.999646   0.782072  0.540936
Literacy:Wealth          1      0.412821     0.412821   0.000861  0.976682
Literacy:Region          4    660.835689   165.208922   0.344548  0.846873
Wealth:Region            4   1276.034369   319.008592   0.665301  0.618369
Literacy:Wealth:Region   4    937.356225   234.339056   0.488720  0.743963
Residuals               65  31167.195053   479.495309        NaN       NaN


# model mixte

Dans R on utilise le package nlme

In [13]:
import statsmodels.datasets
#data = statsmodels.datasets.get_rdataset('dietox', 'geepack').data
data = pd.read_csv('./data/dietox.csv')
data.head()

,Unnamed: 0,Weight,Feed,Time,Pig,Evit,Cu,Litter
0,1,26.50000,NaN,1,4601,1,1,1
1,2,27.59999,5.200005,2,4601,1,1,1
2,3,36.50000,17.600000,3,4601,1,1,1
3,4,40.29999,28.500000,4,4601,1,1,1
4,5,49.09998,45.200001,5,4601,1,1,1


In [14]:
importr('nlme')

fixed = Formula('Weight ~ Time')
#random = Formula('~1|Pig')
random = Formula('~1|Pig')
model = R.lme(fixed=fixed, data=data, random=random)
print(R.summary(model))
# print(R.coef(model))

Linear mixed-effects model fit by REML
 Data: structure(list("Unnamed: 0" = 1:861, Weight = c(26.5, 27.59999,  36.5, 40.29999, 49.09998, 55.39999, 59.59998, 67, 76.59998, 86.5,  91.59998, 98.59998, 27, 31.79999, 39, 44.79999, 50.89999, 57.39999,  62.5, 71.69995, 78.19995, 85.79999, 91.79999, 100.19995, 17,  19, 23.59999, 30, 36.29999, 42.39999, 50.5, 57.59998, 66.59998,  74.19995, 83, 89.29999, 26.89999, 32.29999, 38.59998, 46, 57.79999,  58.59998, 65.89996, 71.19995, 79.39996, 81, 91.69995, 91, 29,  32.69998, 38.29999, 45.5, 48.59998, 56, 62.59998, 70.19995, 73.79999,  80.29999, 85.5, 93.09998, 32, 35.19998, 41.5, 47.69998, 53.79999,  61.59998, 70, 79.09998, 85.69995, 92.5, 101.09998, 109, 21.2,  25.79999, 29.39999, 36.79999, 43.19998, 51.09998, 59.29999, 66,  72.79999, 78.59998, 93.79999, 95.19995, 29.59999, 34.19998, 43,  50, 57, 63.5, 72.69995, 78.69995, 84.89996, 91, 96.39996, 100.19995,  22.39999, 25.2, 30, 36.5, 43.29999, 50, 58.09998, 63, 70.69995,  78.09998, 83.5, 24, 28.2, 35

In [15]:
importr('lme4')

#fixed = Formula('Weight ~ Time')
#random = Formula('~1|Pig')
formula = Formula('Weight ~ Time + (1|Pig)')

model = R.lmer(formula=formula, data=data)
print(R.summary(model))
# print(R.coef(model))

PackageNotInstalledError: The R package "lme4" is not installed.

# Autre solution : uniquement dans jupyter

Utilser la **magic command** %%R qui transforme une cellule en R.

In [16]:
%load_ext rpy2.ipython
%Rpush data

In [17]:
%%R
#Attetion cette cellule c'est du R!!!
library(nlme)
model <- lme(fixed=Weight~Time, data=data, random=~1|Pig)
summary(model)
# coef(model)

Linear mixed-effects model fit by REML
 Data: data 
       AIC      BIC    logLik
  4817.551 4836.574 -2404.775

Random effects:
 Formula: ~1 | Pig
        (Intercept) Residual
StdDev:    6.355624 3.371486

Fixed effects: Weight ~ Time 
                Value Std.Error  DF   t-value p-value
(Intercept) 15.723523 0.7880537 788  19.95235       0
Time         6.942505 0.0333873 788 207.93866       0
 Correlation: 
     (Intr)
Time -0.275

Standardized Within-Group Residuals:
        Min          Q1         Med          Q3         Max 
-4.71175960 -0.56956071 -0.09427981  0.48767373  4.77315870 

Number of Observations: 861
Number of Groups: 72 
